# Step 1 Download Dataset

In [ ]:
!curl -L "https://app.roboflow.com/ds/ePytQzrTOx?key=EW8Eaftnmr" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# เปลี่ยนเป็น Datset ของตนเองได้เลยที่ทำไว้ใน roboflow

# Step 2 Download Ul ultralytics

In [ ]:
!pip install ultralytics

# Step 3 Import Libary and Set env

In [ ]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image


# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))

    all_images.sort()

    num_images = len(all_images)

    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()



# Step 4 Training Model

In [ ]:
# เช็ึคและตรวจสอบเส้นทางของไฟล์ data.yaml

EPOCHS = 150
IMGSIZE = 1280                  # <--- กำหนดค่า config ตรงนี้
BATCHSIZE = 32          
LEARNRATE = 0.01

!yolo task=detect mode=train model=yolov8s.pt imgsz={IMGSIZE} data=/content/drive/MyDrive/AI_Human_Detection/data.yaml epochs={EPOCHS} batch={BATCHSIZE} name=yolov8n_v8_50e lr0={LEARNRATE}


# Step 5 Test Model

In [ ]:
# เช็คตรวจสอบกำหนดเส้นทางไฟล์ best.pt 
# เช็ึคและตรวจสอบเส้นทางของไฟล์ data.yaml

!yolo task=detect mode=predict model=/content/drive/MyDrive/AI_Human_Detection/runs/detect/yolov8n_v8_50e4/weights/best.pt name=yolov8n_predict data=/content/drive/MyDrive/AI_Human_Detection/data.yaml

# Step 6 Predict Video Model ( Using )

In [ ]:
สามารถทำตามไฟล์ main.ipynb ได้แต่ 
แต่ต้องแก้ไข เส้นทางไฟล์ Path model  (/...../.../.../best.pt)